In [1]:
# Local imports
import argparse
import glob
import os
import sys
import re

# Third-party imports
import xarray as xr
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import cftime

In [2]:
# Import the dictionaries
import dictionaries as dic

# Import the functions
import processing_NAO_data as pnd
import functions_lag as fnc_lag
import functions_skill_maps as fnc_sm

In [3]:
# make the saved data directory if it doesn't exist
if not os.path.exists(dic.saved_data_dir):
    os.makedirs(dic.saved_data_dir)

# define the paths to the processed data files
azores_data_path = dic.azores_data_path
iceland_data_path = dic.iceland_data_path

In [4]:
# First thing to do is to process the observations
# We need to set up the parameters for this
forecast_range = "2-6"
season = "DJFM"
variable = "psl"
obs_var_name = "psl"
region_grid = "north-atlantic"
models = dic.models # all models for psl
observations_path = dic.full_era5 # contains all months and variables

In [5]:
# Check if the processed data files exist
if os.path.exists(azores_data_path) and os.path.exists(iceland_data_path):
    # If the files exist, load the processed data from the files
    obs_azores = np.load(azores_data_path)
    obs_iceland = np.load(iceland_data_path)
else:
    # If the files don't exist, process the observations and save the processed data to files
    # Process the observations for the Azores region
    region = "azores"
    obs_azores = fnc_sm.process_observations(variable, region, region_grid,
                                                forecast_range, season, 
                                                observations_path, obs_var_name)
    
    # Now do the same but for the iceland region
    region = "iceland"
    obs_iceland = fnc_sm.process_observations(variable, region, region_grid,
                                            forecast_range, season, 
                                            observations_path, obs_var_name)

    # Save the processed data to files
    np.save(azores_data_path, obs_azores)
    np.save(iceland_data_path, obs_iceland)

In [7]:
obs_azores

array([[[            nan,             nan,             nan,
                     nan],
        [            nan,             nan,             nan,
                     nan]],

       [[            nan,             nan,             nan,
                     nan],
        [            nan,             nan,             nan,
                     nan]],

       [[-3.25855469e+02, -3.18030469e+02, -3.02181250e+02,
         -2.80442187e+02],
        [-3.94129687e+02, -3.87514453e+02, -3.75512500e+02,
         -3.57270703e+02]],

       [[-2.98992969e+02, -2.91889063e+02, -2.80136328e+02,
         -2.61884766e+02],
        [-3.37506641e+02, -3.30225781e+02, -3.18931641e+02,
         -3.02290234e+02]],

       [[-4.19642578e+02, -4.10154297e+02, -3.92710156e+02,
         -3.66444531e+02],
        [-4.82668359e+02, -4.74334375e+02, -4.59581250e+02,
         -4.36519922e+02]],

       [[-3.42625391e+02, -3.30363281e+02, -3.12288281e+02,
         -2.88477344e+02],
        [-3.83682422e+02, -3.7415

In [6]:
# Now we have obs anomaly fields for year 2-6 DJFM psl
# for both the azores and iceland gridboxes
# we want to take the fieldmean of these
# then subtract iceland from azores
# use the calculate_nao_index function from functions_skill_maps.py for this
obs_nao_index = fnc_sm.calculate_nao_index(obs_azores, obs_iceland)

TypeError: _mean() got an unexpected keyword argument 'dim'

In [ ]:
# obs_nao_index.values

In [ ]:
# obs_nao_index.time

In [ ]:
# Now we want to load all of the same init files
# There should be 158 of them (we are missing CanESM5)
# These contain year 2-6 running means
# We will use them for our no lag case
# Set up the models to be included to be the no_CanESM5 models
models = dic.models_no_can

# Set up the base directory
base_dir = dic.base_directory

# Use the function: load_data_same_init from functions_lag.py
# to load the same init files
same_init_members = fnc_lag.load_data_same_init(base_dir, models, 
                                                variable, forecast_range, season)

In [ ]:
# Now we want to process the same init model data before it is plotted
# by extracting the data for out variable of interest (e.g. temperature)
# and the time values
# the function used for this is: process_data from functions_lag.py
processed_same_init_members, same_init_members_time = fnc_lag.process_data(
    same_init_members, variable)

In [ ]:
# # test
# processed_same_init_members['NorCPM1'].time

In [ ]:
# Now we want to plot the no lag case for the 158 members
# Using the plot_ensemble_members_and_mean function
# from processing_NAO_data.py (pnd.)

# Set up the arguments for the function
models = dic.models_no_can
model_times = same_init_members_time
model_nao_anoms = processed_same_init_members
obs_nao_anom = obs_nao_index.values
obs_time = obs_nao_index.time.values

# Run the function
pnd.plot_ensemble_members_and_mean(models, model_times, model_nao_anoms,
                                    obs_nao_anom, obs_time) 